# Azure AI Search: Búsqueda con Tags y Filtros

Este notebook demuestra cómo implementar un sistema de búsqueda que combina:
1. **Búsqueda Vectorial (Semántica)**
2. **Metadatos Estructurados (Tags)**
3. **Filtrado híbrido**

### Requisitos
- Archivo `.env` configurado con:
  - `AZURE_SEARCH_ENDPOINT`
  - `AZURE_SEARCH_KEY`
  - `GITHUB_TOKEN` (o API Key de OpenAI)

## 1. Configuración e Inicialización
Cargamos las variables de entorno e inicializamos los clientes de Azure y OpenAI.

In [8]:
import json
import os
import pathlib
from typing import List, Dict, Any

import openai
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
)
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv(override=True)

# Configuración
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
INDEX_NAME = "documents-with-tags"

# Cliente de OpenAI
openai_client = openai.OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ["GITHUB_TOKEN"]
)

# Clientes de Azure AI Search
credential = AzureKeyCredential(AZURE_SEARCH_KEY)
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=credential)
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=credential)

print("✅ Clientes configurados correctamente")

✅ Clientes configurados correctamente


## 2. Crear el Índice en Azure AI Search
Definimos el esquema del índice. Nota cómo usamos `filterable=True` y `facetable=True` en los campos `category`, `source` y `tags` para permitir el filtrado.

In [9]:
def create_search_index():
    print(f"\n🔧 Creando índice '{INDEX_NAME}'...")

    # Definir los campos del índice
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True),
        SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
        
        # Campos para filtrado
        SimpleField(name="category", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SimpleField(name="source", type=SearchFieldDataType.String, filterable=True, facetable=True),
        # Colección de tags
        SimpleField(name="tags", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=True, facetable=True),
        
        # Campo vectorial
        SearchField(
            name="contentVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_profile_name="myHnswProfile",
        ),
    ]

    # Configurar búsqueda vectorial
    vector_search = VectorSearch(
        profiles=[VectorSearchProfile(name="myHnswProfile", algorithm_configuration_name="myHnsw")],
        algorithms=[HnswAlgorithmConfiguration(name="myHnsw")],
    )

    # Crear el índice
    index = SearchIndex(name=INDEX_NAME, fields=fields, vector_search=vector_search)

    # Eliminar índice si ya existe para empezar de cero
    try:
        index_client.delete_index(INDEX_NAME)
        print(f"  ✓ Índice anterior eliminado")
    except Exception:
        pass

    # Crear nuevo índice
    index_client.create_index(index)
    print(f"  ✓ Índice '{INDEX_NAME}' creado exitosamente")

# Ejecutar la creación
create_search_index()


🔧 Creando índice 'documents-with-tags'...


  ✓ Índice anterior eliminado
  ✓ Índice 'documents-with-tags' creado exitosamente


## 3. Generación de Embeddings e Ingesta de Datos
Definimos los documentos de ejemplo directamente aquí para facilitar la ejecución, generamos sus embeddings y los subimos.

In [11]:
def generate_embedding(text: str) -> List[float]:
    """Genera embedding para un texto usando OpenAI"""
    response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

def index_documents():
    print(f"\n📝 Procesando e indexando documentos...")

    # Datos de ejemplo mockeados para el notebook
    # Obtener la ruta del archivo JSON relativa al directorio actual del notebook
    data_path = pathlib.Path("data/documents_with_metadata.json")
    raw_documents = json.load(open(data_path, "r", encoding="utf-8"))

    docs_to_index = []
    for doc in raw_documents:
        print(f"  Generando embedding para doc {doc['id']} ({doc['category']})...")
        embedding = generate_embedding(doc["content"])
        
        doc_with_embedding = doc.copy()
        doc_with_embedding["contentVector"] = embedding
        docs_to_index.append(doc_with_embedding)

    # Subir documentos
    search_client.upload_documents(documents=docs_to_index)
    print(f"  ✓ {len(docs_to_index)} documentos indexados")

# Ejecutar indexación
index_documents()



📝 Procesando e indexando documentos...
  Generando embedding para doc tech001 (tecnologia)...
  Generando embedding para doc tech002 (tecnologia)...
  Generando embedding para doc tech002 (tecnologia)...
  Generando embedding para doc tech003 (tecnologia)...
  Generando embedding para doc tech003 (tecnologia)...
  Generando embedding para doc tech004 (tecnologia)...
  Generando embedding para doc tech004 (tecnologia)...
  Generando embedding para doc cloud001 (cloud)...
  Generando embedding para doc cloud001 (cloud)...
  Generando embedding para doc cloud002 (cloud)...
  Generando embedding para doc cloud002 (cloud)...
  Generando embedding para doc cloud003 (cloud)...
  Generando embedding para doc cloud003 (cloud)...
  Generando embedding para doc ai001 (ai)...
  Generando embedding para doc ai001 (ai)...
  Generando embedding para doc ai002 (ai)...
  Generando embedding para doc ai002 (ai)...
  Generando embedding para doc ai003 (ai)...
  Generando embedding para doc ai003 (ai)...

## 4. Funciones de Búsqueda
Definimos dos funciones: una para búsqueda solo por filtros (Metadata) y otra para búsqueda híbrida (Semántica + Metadata).

In [12]:
def search_by_tag(category: str = None, source: str = None, tags: List[str] = None):
    """Búsqueda filtrada por tags usando filtros OData"""
    filters = []
    if category: filters.append(f"category eq '{category}'")
    if source: filters.append(f"source eq '{source}'")
    if tags:
        # Sintaxis OData para colecciones: tags/any(t: t eq 'valor')
        tag_filters = [f"tags/any(t: t eq '{tag}')" for tag in tags]
        filters.append(f"({' or '.join(tag_filters)})")

    filter_expression = " and ".join(filters) if filters else None
    print(f"\n🔍 Filtro aplicado: {filter_expression}")

    results = search_client.search(
        search_text="*", filter=filter_expression, select=["id", "content", "category", "tags"], top=3
    )

    for r in results:
        print(f"  - [{r['category']}] {r['content']} (Tags: {r['tags']})")

def hybrid_search(query: str, category: str = None, tags: List[str] = None):
    """Búsqueda híbrida: Vectores + Filtros"""
    query_vector = generate_embedding(query)
    
    filters = []
    if category: filters.append(f"category eq '{category}'")
    if tags:
        tag_filters = [f"tags/any(t: t eq '{tag}')" for tag in tags]
        filters.append(f"({' or '.join(tag_filters)})")
    filter_expression = " and ".join(filters) if filters else None

    print(f"\n🧠 Búsqueda Híbrida: '{query}' | Filtros: {filter_expression}")

    vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=5, fields="contentVector")
    
    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        filter=filter_expression,
        select=["id", "content", "category", "tags"],
        top=3
    )

    for r in results:
        score = r.get('@search.score', 0)
        print(f"  - (Score: {score:.2f}) [{r['category']}] {r['content']}")

## 5. Pruebas y Ejemplos
Ejecuta las celdas siguientes para probar diferentes escenarios.

In [13]:
# Ejemplo 1: Filtrar solo por categoría 'salud'
search_by_tag(category="salud")


🔍 Filtro aplicado: category eq 'salud'
  - [salud] Una dieta mediterránea se caracteriza por un alto consumo de aceite de oliva, legumbres, cereales integrales, frutas y verduras. Se asocia con menor riesgo de enfermedades cardíacas y mayor longevidad. (Tags: ['nutricion', 'dieta', 'salud', 'corazon'])
  - [salud] La meditación mindfulness es una práctica mental que se centra en estar presente en el momento actual. Se ha demostrado que reduce el estrés, mejora la concentración y promueve el bienestar emocional. (Tags: ['meditacion', 'mindfulness', 'salud-mental', 'bienestar'])
  - [salud] La diabetes tipo 2 es una enfermedad crónica que afecta la forma en que el cuerpo metaboliza el azúcar. Con la diabetes tipo 2, el cuerpo no produce suficiente insulina o se resiste a la insulina. (Tags: ['diabetes', 'enfermedad', 'metabolismo'])


In [14]:
# Ejemplo 2: Filtrar documentos que tengan el tag 'ia'
search_by_tag(tags=["ia"])


🔍 Filtro aplicado: (tags/any(t: t eq 'ia'))
  - [ai] Fine-tuning es el proceso de ajustar un modelo preentrenado con datos específicos de un dominio o tarea. Permite adaptar modelos generales a necesidades específicas mejorando su rendimiento en tareas particulares. (Tags: ['fine-tuning', 'ml', 'entrenamiento', 'ia'])
  - [ai] RAG (Retrieval Augmented Generation) es una técnica que combina la recuperación de información con la generación de lenguaje natural. Permite a los modelos de lenguaje acceder a información externa para generar respuestas más precisas y actualizadas. (Tags: ['rag', 'ia', 'llm', 'busqueda'])
  - [ai] Los Large Language Models (LLMs) son modelos de inteligencia artificial entrenados con enormes cantidades de texto. Pueden realizar tareas como generación de texto, traducción, resumen, respuesta a preguntas y programación. (Tags: ['llm', 'ia', 'nlp', 'transformer'])


In [15]:
# Ejemplo 3: Búsqueda Híbrida
# Buscamos sobre 'programación' PERO solo dentro de la categoría 'tecnologia'
hybrid_search("aprender lenguajes de código", category="tecnologia")


🧠 Búsqueda Híbrida: 'aprender lenguajes de código' | Filtros: category eq 'tecnologia'
  - (Score: 0.03) [tecnologia] Python es un lenguaje de programación interpretado de alto nivel y propósito general. Su filosofía de diseño enfatiza la legibilidad del código con el uso de indentación significativa. Python es dinámicamente tipado y cuenta con recolección de basura automática.
  - (Score: 0.03) [tecnologia] JavaScript es un lenguaje de programación ligero, interpretado, orientado a objetos con funciones de primera clase. Es conocido como el lenguaje de scripting para páginas web, pero también se usa en muchos entornos fuera del navegador como Node.js.
  - (Score: 0.03) [tecnologia] Kubernetes es un sistema de código abierto para automatizar el despliegue, escalado y gestión de aplicaciones en contenedores. Fue originalmente diseñado por Google y ahora es mantenido por la Cloud Native Computing Foundation.


In [16]:
# Ejemplo 4: Búsqueda Híbrida con Tags específicos
# Buscamos sobre inteligencia artificial, restringido a documentos con tag 'rag'
hybrid_search("búsqueda de información", tags=["rag"])


🧠 Búsqueda Híbrida: 'búsqueda de información' | Filtros: (tags/any(t: t eq 'rag'))
  - (Score: 0.03) [ai] RAG (Retrieval Augmented Generation) es una técnica que combina la recuperación de información con la generación de lenguaje natural. Permite a los modelos de lenguaje acceder a información externa para generar respuestas más precisas y actualizadas.


In [17]:
# Ejemplo 5: Demostración de filtro excluyente
# Buscamos algo de finanzas pero filtramos por categoría 'salud' -> No debería dar resultados relevantes
hybrid_search("estrategias de inversión", category="salud")


🧠 Búsqueda Híbrida: 'estrategias de inversión' | Filtros: category eq 'salud'
  - (Score: 0.03) [salud] Una dieta mediterránea se caracteriza por un alto consumo de aceite de oliva, legumbres, cereales integrales, frutas y verduras. Se asocia con menor riesgo de enfermedades cardíacas y mayor longevidad.
  - (Score: 0.02) [salud] El ejercicio cardiovascular es cualquier actividad que aumenta el ritmo cardíaco y la respiración. Incluye caminar, correr, nadar y andar en bicicleta. Mejora la salud del corazón y ayuda a controlar el peso.
  - (Score: 0.02) [salud] La meditación mindfulness es una práctica mental que se centra en estar presente en el momento actual. Se ha demostrado que reduce el estrés, mejora la concentración y promueve el bienestar emocional.
